In [1]:
def set_size(width_pt, fraction=1, subplots=(1, 1)):
    """Set figure dimensions to sit nicely in our document.

    Parameters
    ----------
    width_pt: float
            Document width in points
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy
    subplots: array-like, optional
            The number of rows and columns of subplots.
    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    # Width of figure (in pts)
    fig_width_pt = width_pt * fraction
    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    golden_ratio = (5**.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio * (subplots[0] / subplots[1])

    return (fig_width_in, fig_height_in)

## Task 1

In [2]:
import matplotlib as mpl
# Use the pgf backend (must be set before pyplot imported)
mpl.use('pgf')
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.signal import find_peaks

plt.rcParams.update({
    "font.family": "serif",
    "text.usetex": True,
    "pgf.rcfonts": False
})
plt.rcParams['font.size'] = 14

# Define the gaussian function
def gaussian(x, a, b, c):
    return a * np.exp(-((x - b) ** 2) / (2 * c ** 2))

# Initialize an empty list to store the FWHM values
FWHM_values = []

# Loop over the range from 1 to 9 (inclusive)
for i in range(1, 10):
    # Load the data
    data = np.loadtxt(f'Task 1/{i}.TKA', skiprows=2)

    # Create an array for indices
    indices = np.arange(len(data))

    # Find the peaks
    peaks, _ = find_peaks(data, height=2000)

    # Define the window
    window = 20

    # Define the window around the peak
    window_data = data[peaks[0] - window:peaks[0] + window]
    window_indices = indices[peaks[0] - window:peaks[0] + window]

    # Fit the gaussian
    popt, _ = curve_fit(gaussian, window_indices, window_data, p0=[2000, peaks[0], 10])

    # Calculate the FWHM and append it to the list
    print('stdv:', popt[2])
    FWHM = 2 * np.sqrt(2 * np.log(2)) * np.abs(popt[2])
    FWHM_values.append(FWHM)

    # Plot the data and the fitted Gaussian
    plt.figure()
    plt.plot(indices, data, label='Data')
    plt.plot(window_indices, gaussian(window_indices, *popt), label='Fitted Gaussian')
    plt.legend()
    plt.show()

    # Plot the data and the fitted Gaussian
    plt.figure(figsize=set_size(469))
    plt.scatter(window_indices, window_data, label='Data')
    plt.plot(window_indices, gaussian(window_indices, *popt), label='Fitted Gaussian')
    plt.legend()
    plt.savefig(f'Latex\Figures\Task 1\{i}.pgf', format='pgf', bbox_inches = 'tight')
    plt.show()

# Convert the list to a numpy array
FWHM_values = np.array(FWHM_values)

print('FWHM values:', FWHM_values)

stdv: 9.142215766558298


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_33832\3644040061.py:54: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_33832\3644040061.py:62: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


stdv: 6.442688669130202
stdv: -4.683474828506527
stdv: 2.934873678694544
stdv: 2.353977451050651
stdv: 2.011387383822805
stdv: 1.8561409445744688
stdv: 1.8307284760839473
stdv: 1.8488425046350008
FWHM values: [21.52827294 15.17137242 11.02874041  6.91109937  5.54319329  4.73645533
  4.3708779   4.31103611  4.35369139]


In [3]:
# Define the rise times
rise_times = np.array([0.5, 0.8, 1.2, 2.4, 3.6, 5.6, 8, 10.4, 13.6])

# Plot the data and the fitted exponential, with the R^2 value and the parameters
plt.figure(figsize=set_size(469))
plt.scatter(rise_times, FWHM_values)

plt.xlabel('Rise Time')
plt.ylabel('FWHM')
plt.title('FWHM vs Rise Time')
plt.grid(True)
plt.savefig('Latex\Figures\Task 1\Risetime.pgf', format='pgf', bbox_inches = 'tight')
plt.show()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_33832\3166436931.py:13: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## Task 2

In [4]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import numpy as np



# Define the path to the folder containing the .TKA files
folder_path = r'Task 2'

# Use glob to find all .TKA files in the specified folder
file_pattern = os.path.join(folder_path, '*.TKA')
tka_files = glob.glob(file_pattern)

# Initialize a list to store dataframes
dataframes = []

# Load each .TKA file into a pandas DataFrame and store it in the list
for file in tka_files:
    df = pd.read_csv(file, delimiter='\t', skiprows=2)  # Skip the first two rows
    dataframes.append((os.path.basename(file), df))

# Peak energies for each element
peak_energies = {
    'Cs137': [661.657],
    'Ba133': [80.9979, 160.6129, 223.2368, 276.3989, 302.8508, 356.0129, 383.8484, 437.0108],
    'Co60': [1173.228, 1332.492],
    'Na22': [511.0, 1274.537]  
}

# Define colors for each element
element_colors = {
    'Cs137': 'blue',
    'Ba133': 'purple',
    'Co60': 'green',
    'Na22': 'red'    
}

# Prepare lists for plotting channel number vs energy
channel_numbers = []
energies = []
colors = []
labels = []

# Plot the data from each DataFrame
for filename, df in dataframes:
    fig, ax = plt.subplots()
    fig.set_size_inches(set_size(469))  # Set the size of the figure (width, height) in inches
    
    for column in df.columns:
        # Normalize the data
        normalized_data = df[column] / df[column].max()
        
        ax.plot(normalized_data, label=column)
        
        # Detect peaks in the normalized data
        if 'Ba133' in filename:
            peaks, _ = find_peaks(normalized_data, prominence=0.01)
        else:
            peaks, _ = find_peaks(normalized_data, prominence=0.12)
        
        # Remove the first peak for Na and Co if needed
        element_name = os.path.splitext(filename)[0]
        if 'Na' in element_name or 'Co' in element_name or 'Ba' in element_name:
            peaks = peaks[1:]
        
        # Get corresponding energies from the dictionary
        peak_energy_list = peak_energies.get(element_name, [])
        
        # Collect channel numbers, energies, colors, and labels for the final plot
        for i, peak in enumerate(peaks):
            if i < len(peak_energy_list):
                channel_numbers.append(peak)
                energies.append(peak_energy_list[i])
                colors.append(element_colors.get(element_name, 'black'))
                labels.append(element_name)
            
            ax.annotate(f'{peak}', 
                        (peak, normalized_data.iloc[peak]), 
                        textcoords="offset points", 
                        xytext=(0,2), 
                        ha='center', fontsize=10)
    
    # Extract element symbol and atomic number from filename, removing the .TKA extension
    base_filename = os.path.splitext(filename)[0]
    element_symbol = ''.join(filter(str.isalpha, base_filename))
    atomic_number = ''.join(filter(str.isdigit, base_filename))
    
    # Format title in LaTeX
    title = f'$^{{{atomic_number}}}\\mathrm{{{element_symbol}}}$'
    
    
    ax.set_title(f'{title}')
    ax.set_xlabel('Channel Number [a.u.]')  
    ax.set_ylabel('Normalized Counts [a.u.]')  
    
    # Set x-axis limits for Ba133
    if 'Ba133' in filename:
        ax.set_xlim(0, 2000)
    
    plt.savefig(f'Latex\Figures\Task 2\{base_filename}.pgf', format='pgf', bbox_inches = 'tight')
    
    plt.show()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_33832\3577940148.py:105: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_33832\3577940148.py:49: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots()


In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

# Function to convert labels to LaTeX format
def format_label(label):
    element_symbol = ''.join(filter(str.isalpha, label))
    atomic_number = ''.join(filter(str.isdigit, label))
    return f'$^{{{atomic_number}}}${element_symbol}'

# Define a linear function for curve fitting
def linear_func(x, a, b):
    return a * x + b

# Convert channel_numbers to a NumPy array
channel_numbers = np.array(channel_numbers)

# Perform linear regression using scipy curve_fit
popt, _ = curve_fit(linear_func, channel_numbers, energies)
slope, intercept = popt

# Calculate R^2 value using sklearn
y_fit = linear_func(channel_numbers, *popt)
r_squared = r2_score(energies, y_fit)

# Generate x values for the linear fit line
x_fit = np.linspace(min(channel_numbers), max(channel_numbers), 100)
y_fit_line = linear_func(x_fit, *popt)

# Plot Channel Number vs Energy with the linear fit
fig, ax = plt.subplots()
added_labels = {}
element_handles = []
element_labels = []

# Plot the line of best fit
line_handle, = ax.plot(x_fit, y_fit_line, '--', label=f'$E = {slope:.3f}C + {intercept:.2f}$ \n $R^2 = {r_squared:.7f}$', color='black')

# Plot the data points
for i in range(len(channel_numbers)):
    formatted_label = format_label(labels[i])
    label = formatted_label if formatted_label not in added_labels else "_nolegend_"
    point_handle, = ax.plot(channel_numbers[i], energies[i], 'o', color=colors[i], label=label, markersize=5)
    if formatted_label not in added_labels:
        element_handles.append(point_handle)
        element_labels.append(formatted_label)
    added_labels[formatted_label] = True

# Reorder the elements in the legend
# 1st -> 3rd, 3rd -> 4th, 4th -> 1st
reordered_indices = [3, 1, 0, 2] + list(range(4, len(element_handles)))
element_handles = [element_handles[i] for i in reordered_indices]
element_labels = [element_labels[i] for i in reordered_indices]

# Create the first legend for the elements
element_legend = ax.legend(element_handles, element_labels, loc='upper left', title='Elements')

# Add the first legend to the plot
ax.add_artist(element_legend)

# Create the second legend for the line of best fit
ax.legend([line_handle], [line_handle.get_label()], loc='lower right', title='Fit')

ax.set_title('Energy Calibration')
ax.set_xlabel('Channel Number [a.u.]')
ax.set_ylabel('Energy [keV]')
fig.set_size_inches(set_size(469))
plt.savefig('Latex\Figures\Task 2\EnergyCalibration.pgf', format='pgf', bbox_inches = 'tight')
plt.show()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_33832\3214339988.py:70: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## Task 3

In [6]:
import numpy as np
import matplotlib.pyplot as plt

# Load the data from Task 3
background = np.loadtxt('Task 3/Background.TKA', skiprows=2)

# Normalize the data
normalized_data = background / background.max()

# Calibrate x axis based on fit 
calibrated_x = linear_func(np.arange(len(data)), *popt)


# Plot the data
plt.figure()
plt.plot(calibrated_x, normalized_data)

plt.xlabel('Energy [keV]')
plt.ylabel('Normalized Counts [a.u.]')
plt.title('Background Spectrum')
plt.legend()
plt.savefig('Latex\Figures\Task 3\Background.pgf', format='pgf', bbox_inches = 'tight')
plt.show()



No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_33832\2508008101.py:23: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [7]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

# Load the data from Task 3
data = np.loadtxt('Task 3/Background.TKA', skiprows=2)

# Define the window range
window_start = 100
window_end = 8190

# Filter the data within the specified window
filtered_data = data[window_start:window_end]

# Normalize the filtered data
normalized_data = filtered_data / filtered_data.max()

# Calibrate x axis based on fit 
calibrated_x = linear_func(np.arange(window_start, window_end), *popt)

# Create a prominence array that decreases linearly
prominence_start = 0.19
prominence_end = 0.02
prominence_array = np.geomspace(prominence_start, prominence_end, len(normalized_data))

# Find peaks in the normalized data with varying prominence
peaks, _ = find_peaks(normalized_data, prominence=prominence_array)

# Remove the first 4 peaks
peaks = peaks[4:]

peak_energies = calibrated_x[peaks]
print("Energies of detected peaks:", peak_energies)

# Define isotopes and their energies from the table
isotopes = [
    (2618.58, '$^{208}$Tl'),
    (1766.32, '$^{214}$Bi'),
    (1461.58, '$^{40}$K'),
    (1121.32, '$^{214}$Bi'),
    (911.49, '$^{228}$Ac'),
    (609.65, '$^{214}$Bi'),
    (583.50, '$^{208}$Tl'),
    (511.19, 'Annihilation'),
    (352.36, '$^{214}$Pb'),
    (295.54, '$^{214}$Pb'),
    (239.05, '$^{212}$Pb'),
    (209.67, '$^{228}$Ac'),
    (186.43, '$^{226}$Ra')
]

# Map isotopes to peaks
label_positions = {}
for energy, label in isotopes:
    closest_peak_idx = (np.abs(calibrated_x - energy)).argmin()
    if label not in label_positions:
        label_positions[label] = []
    label_positions[label].append(closest_peak_idx)

# Plot the data
plt.figure(figsize=set_size(469*1.5))
plt.plot(calibrated_x, normalized_data)
plt.plot(calibrated_x[peaks], normalized_data[peaks], ".", color='red', markersize=5)

# Annotate the isotopes with a single label pointing to all corresponding peaks
label_text_positions = {
    '$^{208}$Tl': (2000, 0.8),
    '$^{214}$Bi': (1950, 0.5),
    '$^{40}$K': (1900, 0.7),
    '$^{228}$Ac': (850, 0.9),
    'Annihilation': (550, 0.55),
    '$^{214}$Pb': (350, 0.65),
    '$^{212}$Pb': (500, 0.95),
    '$^{226}$Ra': (500, 0.9)
}

# Define colors for each label
colors = {
    '$^{208}$Tl': 'blue',
    '$^{214}$Bi': 'green',
    '$^{40}$K': 'indigo',
    '$^{228}$Ac': 'purple',
    'Annihilation': 'teal',
    '$^{214}$Pb': 'brown',
    '$^{212}$Pb': 'magenta',
    '$^{226}$Ra': 'red'
}

# Annotate the isotopes with colored arrows
for label, indices in label_positions.items():
    color = colors.get(label, 'black')  # Default to black if label not in colors
    label_x, label_y = label_text_positions.get(label, (0, 0))
    for idx in indices:
        if 0 <= idx < len(calibrated_x):  # Ensure index is within bounds
            plt.annotate('', xy=(calibrated_x[idx], normalized_data[idx]), xytext=(label_x, label_y),
                         arrowprops=dict(arrowstyle='->', color=color))
    plt.text(label_x, label_y, label, fontsize=9, ha='center', color=color)

plt.xlabel('Energy [keV]')
plt.ylabel('Normalized Counts [a.u.]')
plt.title('Background Spectrum (Windowed)')
plt.savefig('Latex\Figures\Task 3\BackgroundWindowed.pgf', format='pgf', bbox_inches = 'tight')
plt.show()

Energies of detected peaks: [ 186.42607749  209.6693528   239.04627021  295.54034216  352.35723738
  511.18628537  583.49869746  609.64738219  911.48713803 1121.32226241
 1461.57798717 1766.32315242 2618.57658066]


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_33832\829054457.py:103: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## Task 4

In [8]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

# Load the background data
background = np.loadtxt('Task 3/Background.TKA', skiprows=2)
normalized_background = background / 3600

# Get all .TKA files in the Task 4 directory
task4_files = glob.glob('Task 4/*.TKA')

# Define isotopes, label positions, and colors for each file
file_annotations = {
    'Chernobyl': {
        'isotopes': [
            (2616.64, '$^{208}$Tl'),
            (238.72, '$^{212}$Pb'),
            (208.70, '$^{228}$Ac'),
            (1461.90, '$^{40}$K'),
            (661.94, '$^{137}$Cs'),
            (131.22, '$^{234}$Pa')
        ],
        'label_positions': {
            '$^{208}$Tl': (2500, 0.04),
            '$^{212}$Pb': (400, 0.06),
            '$^{228}$Ac': (200, 0.07),
            '$^{40}$K': (1400, 0.04),
            '$^{137}$Cs': (1000, 0.125),
            '$^{234}$Pa': (90, 0.09)
        },
        'colors': {
            '$^{208}$Tl': 'blue',
            '$^{212}$Pb': 'green',
            '$^{228}$Ac': 'red',
            '$^{40}$K': 'magenta',
            '$^{137}$Cs': 'orange',
            '$^{234}$Pa': 'cyan'
        }
    },
    'Soil': {
        'isotopes': [
            (1766.32, '$^{214}$Bi'),
            (1239.15, '$^{214}$Bi'),
            (1121.32, '$^{214}$Bi'),
            (609.97, '$^{214}$Bi'),
            (352.36, '$^{214}$Pb'),
            (295.54, '$^{214}$Pb'),
            (242.27, '$^{214}$Pb'),
            (186.43, '$^{226}$Ra'),
            (1462.22, '$^{40}$K'),
            (77.63, 'Bi K$\\alpha_{1}$')
        ],
        'label_positions': {
            '$^{214}$Bi': (700, 0.5),
            '$^{214}$Pb': (100, 0.64),
            '$^{226}$Ra': (60, 0.45),
            '$^{40}$K': (1200, 0.2),
            'Bi K$\\alpha_{1}$': (30, 0.3)
        },
        'colors': {
            '$^{214}$Bi': 'blue',
            '$^{214}$Pb': 'green',
            '$^{226}$Ra': 'red',
            '$^{40}$K': 'magenta',
            'Bi K$\\alpha_{1}$': 'orange'
        }
    },
    'Rock': {
        'isotopes': [
            (1766.32, '$^{214}$Bi'),
            (1120.67, '$^{214}$Bi'),
            (609.32, '$^{214}$Bi'),
            (352.36, '$^{214}$Pb'),
            (295.54, '$^{214}$Pb'),
            (242.27, '$^{214}$Pb'),
            (186.43, '$^{226}$Ra'),
            (92.81, '$^{234}$Th'),
            (77.31, 'Bi K$\\alpha_{1}$')
        ],
        'label_positions': {
            '$^{214}$Bi': (900, 1.6),
            '$^{214}$Pb': (200, 2.3),
            '$^{226}$Ra': (110, 1.3),
            '$^{234}$Th': (50, 0.9),
            'Bi K$\\alpha_{1}$': (0, 1.2)
        },
        'colors': {
            '$^{214}$Bi': 'blue',
            '$^{214}$Pb': 'green',
            '$^{226}$Ra': 'red',
            '$^{234}$Th': 'cyan',
            'Bi K$\\alpha_{1}$': 'orange'
        }
    }
}


for file in task4_files:
    # Load the data from the current file
    data = np.loadtxt(file, skiprows=2)
    
    # Normalize the data
    if 'Chernobyl' in file:
        normalized_data = data / 600
    else:
        normalized_data = data / 180
    
    # Calibrate the x-axis
    calibrated_x = linear_func(np.arange(len(data)), *popt)
    
    # Subtract the background
    subtracted_data = normalized_data - normalized_background
    
    # Identify peaks
    if 'Chernobyl' in file:
        prominence_start = 0.07
        prominence_end = 0.005
    elif 'Soil' in file:
        prominence_start = 0.15
        prominence_end = 0.015
    else:
        prominence_start = 0.4
        prominence_end = 0.05
    
    prominence_array = np.geomspace(prominence_start, prominence_end, len(subtracted_data))
    peaks, _ = find_peaks(subtracted_data, prominence=prominence_array)
    
    # Special peak removal logic for specific files
    if 'Chernobyl' in file:
        peaks = peaks[:-2]
        peaks = np.delete(peaks, -2)
        peaks = np.delete(peaks, -2)
        peaks = peaks[4:]
    elif 'Soil' in file:
        peaks = np.delete(peaks, 0)
        peaks = np.delete(peaks, 1)
        peaks = np.delete(peaks, 2)
        peaks = np.delete(peaks, 1)
    elif 'Rock' in file:
        peaks = peaks[1:]
        
    peak_energies = calibrated_x[peaks]
    print(f'Peaks for {os.path.basename(file)}: {peak_energies}')
    
    # Determine the correct annotations based on the file type
    basename = os.path.basename(file)
    file_type = None
    for key in file_annotations:
        if key in basename:
            file_type = key
            break
    
    if file_type:
        isotopes = file_annotations[file_type]['isotopes']
        label_positions = file_annotations[file_type]['label_positions']
        colors = file_annotations[file_type]['colors']
        
        # Map isotopes to peaks
        label_positions_peaks = {}
        for energy, label in isotopes:
            closest_peak_idx = (np.abs(calibrated_x - energy)).argmin()
            if label not in label_positions_peaks:
                label_positions_peaks[label] = []
            label_positions_peaks[label].append(closest_peak_idx)
        
        # Plot the data
        plt.figure(figsize=set_size(469))
        plt.plot(calibrated_x, np.abs(subtracted_data))
        plt.plot(calibrated_x[peaks], np.abs(subtracted_data)[peaks], ".", color='red', markersize=5)
        plt.xlabel('Energy [keV]')
        plt.ylabel('Count Rate [Counts/s]')
        if 'Rock' in file:
            sample= ' - Ronneb 1'
        
        elif 'Soil' in file:
            sample= ' - QQ250m'

        else:
            sample = ''

        plt.title(f'{os.path.splitext(os.path.basename(file))[0]} Sample Spectrum' + sample)
        
        # Annotate the isotopes with colored arrows
        for label, indices in label_positions_peaks.items():
            color = colors.get(label, 'black')
            label_x, label_y = label_positions.get(label, (0, 0))
            for idx in indices:
                if 0 <= idx < len(calibrated_x):  # Ensure index is within bounds
                    plt.annotate('', xy=(calibrated_x[idx], np.abs(subtracted_data)[idx]), xytext=(label_x, label_y),
                                 arrowprops=dict(arrowstyle='->', color=color))
            plt.text(label_x, label_y, label, fontsize=9, ha='center', color=color)
        plt.savefig(f'Latex\Figures\Task 4\{os.path.splitext(os.path.basename(file))[0]}.pgf', format='pgf', bbox_inches = 'tight')
        plt.show()



Peaks for Chernobyl.TKA: [ 131.22329861  208.700883    238.72344695  661.94475165 1461.90081044
 2616.63964105]


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_33832\2355945038.py:195: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


Peaks for Rock.TKA: [  77.31181281   92.80732969  186.42607749  242.2745029   295.54034216
  352.35723738  609.32455892 1120.67661587 1766.32315242]
Peaks for Soil.TKA: [  77.63463608  186.42607749  242.2745029   295.54034216  352.35723738
  609.97020546 1121.32226241 1239.15275533 1462.2236337  1766.32315242]


## Task 5

In [9]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

# Load the background data
background = np.loadtxt('Task 3/Background.TKA', skiprows=2)
normalized_background = background / 3600

# Get all .TKA files in the Task 4 directory
task4_files = glob.glob('Task 5/*.TKA')

# Define isotopes, label positions, and colors for each file
file_annotations = {
    'W': {
        'isotopes': [
            (2618.25, '$^{208}$Tl'),
            (969.60, '$^{228}$Ac'),
            (911.16, '$^{228}$Ac'),
            (583.50, '$^{208}$Tl'),
            (338.48, '$^{228}$Ac'),
            (239.05, '$^{212}$Pb'),
            (210.00, '$^{228}$Ac'),
            (77.63, 'Bi K$\\alpha_{1}$')
        ],
        'label_positions': {
            '$^{208}$Tl': (1700, 1),
            '$^{212}$Pb': (400, 2.3),
            '$^{228}$Ac': (700, 1),
            'Bi K$\\alpha_{1}$': (100, 1),
        },
        'colors': {
            '$^{208}$Tl': 'blue',
            '$^{212}$Pb': 'green',
            '$^{228}$Ac': 'red',
            'Bi K$\\alpha_{1}$': 'orange',
        }
    },
    'V': {
        'isotopes': [
            (1766.32, '$^{214}$Bi'),
            (1120.68, '$^{214}$Bi'),
            (609.65, '$^{214}$Bi'),
            (352.36, '$^{214}$Pb'),
            (295.54, '$^{214}$Pb'),
            (242.60, '$^{214}$Pb'),
            (186.43, '$^{226}$Ra'),
            (93.13, '$^{234}$Th')
        ],
        'label_positions': {
            '$^{214}$Bi': (1200, 4),
            '$^{214}$Pb': (500, 6),
            '$^{226}$Ra': (60,4.5),
            '$^{234}$Th': (50, 3),
        },
        'colors': {
            '$^{214}$Bi': 'blue',
            '$^{214}$Pb': 'green',
            '$^{226}$Ra': 'red',
            '$^{234}$Th': 'magenta',
        }
    } 
}


for file in task4_files:
    # Load the data from the current file
    data = np.loadtxt(file, skiprows=2)
    
    
    normalized_data = data / 180
    
    # Calibrate the x-axis
    calibrated_x = linear_func(np.arange(len(data)), *popt)
    
    # Subtract the background
    subtracted_data = normalized_data - normalized_background
    
    # Identify peaks
    if 'V' in file:
        prominence_array = np.geomspace(1.8, 0.05, len(subtracted_data))
    else:
        prominence_array = np.geomspace(0.3, 0.02, len(subtracted_data))

    peaks, _ = find_peaks(np.abs(subtracted_data), prominence=prominence_array)

    if 'V' in file:
        peaks = peaks[1:]
        peaks = np.delete(peaks, 1)
    else:
        peaks = peaks[2:]
        peaks = np.delete(peaks, 1)
        peaks = np.delete(peaks, 1)
        

        
    peak_energies = calibrated_x[peaks]
    print(f'Peaks for {os.path.basename(file)}: {peak_energies}')
    
    # Determine the correct annotations based on the file type
    basename = os.path.basename(file)
    file_type = None
    for key in file_annotations:
        if key in basename:
            file_type = key
            break
    
    if file_type:
        isotopes = file_annotations[file_type]['isotopes']
        label_positions = file_annotations[file_type]['label_positions']
        colors = file_annotations[file_type]['colors']
        
        # Map isotopes to peaks
        label_positions_peaks = {}
        for energy, label in isotopes:
            closest_peak_idx = (np.abs(calibrated_x - energy)).argmin()
            if label not in label_positions_peaks:
                label_positions_peaks[label] = []
            label_positions_peaks[label].append(closest_peak_idx)
        
        # Plot the data
        plt.figure(figsize=set_size(469))
        plt.plot(calibrated_x, np.abs(subtracted_data))
        plt.plot(calibrated_x[peaks], np.abs(subtracted_data)[peaks], ".", color='red', markersize=5)
        plt.xlabel('Energy [keV]')
        plt.ylabel('Count Rate [Counts/s]')
        plt.title(f'{os.path.splitext(os.path.basename(file))[0]}-Sample Spectrum')
        
        # Annotate the isotopes with colored arrows
        for label, indices in label_positions_peaks.items():
            color = colors.get(label, 'black')
            label_x, label_y = label_positions.get(label, (0, 0))
            for idx in indices:
                if 0 <= idx < len(calibrated_x):  # Ensure index is within bounds
                    plt.annotate('', xy=(calibrated_x[idx], np.abs(subtracted_data)[idx]), xytext=(label_x, label_y),
                                 arrowprops=dict(arrowstyle='->', color=color))
            plt.text(label_x, label_y, label, fontsize=9, ha='center', color=color)
        plt.savefig(f'Latex\Figures\Task 5\{os.path.splitext(os.path.basename(file))[0]}.pgf', format='pgf', bbox_inches = 'tight')
        plt.show()



Peaks for V.TKA: [  93.13015295  186.42607749  242.59732617  295.54034216  352.35723738
  609.64738219 1120.67661587 1766.32315242]
Peaks for W.TKA: [  77.63463608  209.99217607  239.04627021  338.47583684  583.49869746
  911.16431476  969.59532632 2618.25375739]


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_33832\2861199129.py:140: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
